In [1]:
from sympy import symbols, pi

from robot import Robot, JointType

from math_utils import Vector, Matrix

In [2]:
d1, d2, d3, d4 = symbols('d1 d2 d3 d4')
t1, t2, t3, t4 = symbols('t1 t2 t3 t4')
l1, l2, l3, l4 = symbols('l1 l2 l3 l4')
a1, a2, a3, a4 = symbols('a1 a2 a3 a4')
s1, s2, s3, s4 = symbols('s1 s2 s3 s4')
g = symbols('g')

In [3]:
# dh = [
#     [0, 0, s1, t1],
#     [a1, -pi/2, s2, t2],
#     [a2,0,s3,t3],
#     [a3, 0, s4, 0]
# ]
dh = [
    [0, 0, l1, t1],
    [0, pi/2, 0, t2],
    [0,pi/2,l2+d3,0],
    [0, 0, 0, 0]
]
robot = Robot.from_dh_parameters(dh, '2rp')

In [4]:
# print Transformations
robot.get_transformations()

Matrix[
	cos(t1),    -sin(t1),    0,    0,    
	sin(t1),    cos(t1),    0,    0,    
	0,    0,    1,    l1,    
	0,    0,    0,    1,    
]
Matrix[
	cos(t2),    -sin(t2),    0,    0,    
	0,    0,    -1,    0,    
	sin(t2),    cos(t2),    0,    0,    
	0,    0,    0,    1,    
]
Matrix[
	1,    0,    0,    0,    
	0,    0,    -1,    -d3 - l2,    
	0,    1,    0,    0,    
	0,    0,    0,    1,    
]
Matrix[
	1,    0,    0,    0,    
	0,    1,    0,    0,    
	0,    0,    1,    0,    
	0,    0,    0,    1,    
]


In [5]:
# print Rotations
robot.get_rotations()

Matrix[
	cos(t1),    -sin(t1),    0,    
	sin(t1),    cos(t1),    0,    
	0,    0,    1,    
]
Matrix[
	cos(t2),    -sin(t2),    0,    
	0,    0,    -1,    
	sin(t2),    cos(t2),    0,    
]
Matrix[
	1,    0,    0,    
	0,    0,    -1,    
	0,    1,    0,    
]
Matrix[
	1,    0,    0,    
	0,    1,    0,    
	0,    0,    1,    
]


In [6]:
# end-effector position in zero frame
p_e = (robot.get_end_transformation() @ Vector([0, 0, 0, 1])).simplify()
print(p_e.subs({t1: 0, t2: pi/4}))

Vector[	sqrt(2)*(d3 + l2)/2,  0,  l1 - sqrt(2)*(d3 + l2)/2,  1,  ]


In [ ]:
# get/ print Z axis in zero fram
axis = robot.get_zero_Z_axis(True)

In [30]:
# calculate velocities
ws, vs = robot.cal_velocities(True)
for i,e in enumerate(vs[1:]):
    print(robot.rotate_to_zero_from_i(e, i))

joint 1:
 - w: Vector[	0,  0,  dot_t1,  ]
 - v: Vector[	0,  0,  0,  ]
joint 2:
 - w: Vector[	dot_t1*sin(t2),  dot_t1*cos(t2),  dot_t2,  ]
 - v: Vector[	0,  0,  0,  ]
joint 3:
 - w: Vector[	dot_t1*sin(t2),  dot_t2,  -dot_t1*cos(t2),  ]
 - v: Vector[	dot_t2*(d3 + l2),  -dot_t1*(d3 + l2)*sin(t2),  dot_d3,  ]
joint 4:
 - w: Vector[	dot_t1*sin(t2),  dot_t2,  -dot_t1*cos(t2),  ]
 - v: Vector[	dot_t2*(d3 + l2),  -dot_t1*(d3 + l2)*sin(t2),  dot_d3,  ]


TypeError: 'NoneType' object is not iterable

In [ ]:
# calculate static kinematics
ws, vs, fs, ns, ts = robot.cal_static_kinematics(True, True)

In [16]:
# calculate dynamic newton euler kinematics
# define center of mass points for each joint
cof_points = [Vector([0, 0, 0]), Vector([l1 , 0, 0]), Vector([0, 0, 0])]
robot.add_cof(cof_points)
# run calculation of newton euler
stuff = robot.cal_newton_euler(True, True, Vector([0, g, 0]))

----------------Forwards Newton Euler----------------



joint 1:
 - w: Vector[	0,  0,  dot_t1,  ]
 - dw: Vector[	0,  0,  ddot_t1,  ]
 - dv: Vector[	g*sin(t1),  g*cos(t1),  0,  ]
simplified 1:
 - w: Vector[	0,  0,  dot_t1,  ]
 - dw: Vector[	0,  0,  ddot_t1,  ]
 - dv: Vector[	g*sin(t1),  g*cos(t1),  0,  ]
joint 1:
 - dvc: Vector[	g*sin(t1),  g*cos(t1),  0,  ]
 - F: Vector[	g*m1*sin(t1),  g*m1*cos(t1),  0,  ]
 - N: Vector[	0,  0,  I_zz1*ddot_t1,  ]
simplified 1:
 - dvc: Vector[	g*sin(t1),  g*cos(t1),  0,  ]
 - F: Vector[	g*m1*sin(t1),  g*m1*cos(t1),  0,  ]
 - N: Vector[	0,  0,  I_zz1*ddot_t1,  ]

joint 2:
 - w: Vector[	dot_t1*sin(t2),  dot_t1*cos(t2),  dot_t2,  ]
 - dw: Vector[	ddot_t1*sin(t2) + dot_t1*dot_t2*cos(t2),  ddot_t1*cos(t2) - dot_t1*dot_t2*sin(t2),  ddot_t2,  ]
 - dv: Vector[	(ddot_t1*s2 + g*sin(t1))*cos(t2),  -(ddot_t1*s2 + g*sin(t1))*sin(t2),  -dot_t1**2*s2 - g*cos(t1),  ]
simplified 2:
 - w: Vector[	dot_t1*sin(t2),  dot_t1*cos(t2),  dot_t2,  ]
 - dw: Vector[	ddot_t1*sin(t2)

In [27]:
# translate vector from specific frame
robot.transform_to_zero_from_i(Vector([0, 0, 0, 1]), 4).simplify()
# rotate vector from specific frame
print(robot.rotate_to_zero_from_i(Vector([0, 0, symbols('ddot_d3')]), 3).simplify())

Vector[	ddot_d3*sin(t2)*cos(t1),  ddot_d3*sin(t1)*sin(t2),  -ddot_d3*cos(t2),  ]
